In [31]:
from time import sleep

from credentials import credentials
from TwitterAPI import TwitterAPI

In [11]:
creds = credentials.require(['access_token', 
                             'access_token_secret', 
                             'consumer_key',
                             'consumer_secret'])

In [13]:
api = TwitterAPI(creds.consumer_key,
                 creds.consumer_secret,
                 creds.access_token,
                 creds.access_token_secret)

In [22]:
def tweets():
    "A generator of tweets"
    while True:
        try:
            iterator = api.request('search/tweets', {'q':'pizza'}).get_iterator()
            for item in iterator:
                if 'text' in item:
                    yield(item['text'])
                elif 'disconnect' in item:
                    event = item['disconnect']
                    if event['code'] in [2,5,6,7]:
                        # something needs to be fixed before re-connecting
                        raise Exception(event['reason'])
                    else:
                        # temporary interruption, re-try request
                        break
        except TwitterRequestError as e:
            if e.status_code < 500:
                # something needs to be fixed before re-connecting
                raise
            else:
                # temporary interruption, re-try request
                pass
        except TwitterConnectionError:
            # temporary interruption, re-try request
            pass

In [35]:
for _ in range(10):
    print(next(t), end='\n'+'#'*20+"\n")
    sleep(1)

RT @LifeAsADabber: When the whole squad high af and the pizza guys shows up at the door https://t.co/l6ZXNhPWaV
####################
Does it make me a fatass if I can eat an entire blaze pizza on my own .? 😐 lol
####################
RT @ItsDanniRyan: Screw sleep, I need pizza 😂 #pimpmysummerballteesside https://t.co/1RW8TgEgFf
####################
RT @youfunnyb: Wake up &amp; remembered you have left over pizza #youfunnyb https://t.co/MC9144EJ24
####################
RT @WorIdStarVine: When your mom says no pizza rolls but you put them in the cart anyway https://t.co/s2gdaLLhbO
####################
Me: "r u looking at my stomach?"
@jakeburnstein : "no, u don't have 1"
Me: "watch me fill it w/ pizza"  🍕
####################
@mone0115 
最高です！
1Dは少ししてからでした😊
どっちも素敵ですよね💗
ありがとうございます！
いえいえそんな🌟
####################
Not just talking bout pizza.
####################
I'm at California Pizza Kitchen - @cpkmexico in Naucalpan, MEX https://t.co/xggg3CGid2
####################
pizza食べ放題でいっぱい食べた(｀・∞・´)
